In [1]:
import json
import pandas as pd
import tweepy

import time

In [2]:
with open("twitter_credentials.json", "r") as file:
    data = file.read()
    
credentials = json.loads(data)
client = tweepy.Client(bearer_token=credentials["BEARER_TOKEN"])

In [3]:
seed_account = 'BenjaminNorton'
seed = client.get_user(username = seed_account)

In [10]:
def get_following(client, user_ids):
    following_list = []
    
    for user_id in user_ids:
        try:
            f = client.get_users_following(user_id, max_results = 1000).data
            if f:
                following_list.extend(f) # for more than 1000 results return, need to use pagination token
        except tweepy.TweepyException:
            print(str(user_id) + "error fetching following list")
            continue
            
    return following_list

In [77]:
f = get_following(client, [seed.data.id])

In [85]:
following_graph = pd.DataFrame(columns=['source','target'])

following_graph['target'] = [user.id for user in f]
following_graph['source'] = seed.data.id

In [86]:
following_graph

,source,target
0,852269288,1116129122
1,852269288,192547775
2,852269288,1295505639926640641
3,852269288,1167697135359287296
4,852269288,50134348
...,...,...
569,852269288,20295264
570,852269288,1051979395
571,852269288,37501003
572,852269288,15721256


In [87]:
starting_users = following_graph['target']

for user in starting_users:
    following = get_following(client, [user])
    
    temp = pd.DataFrame(columns=['source','target'])
    temp['target'] = [f.id for f in following]
    temp['source'] = user
    
    following_graph = following_graph.append(temp)
    following_graph.to_csv('following_list.csv', index=False)
    print(str(user) + " added")
    
    time.sleep(60)

1116129122 added
192547775 added
1295505639926640641 added
1167697135359287296 added
50134348 added
139049986 added
189365211 added
1207282997000577024 added
83552557 added
23047519 added
772368400035278848 added
18000989 added
1212564056827031553 added
1345884115346870272 added
961925545809281024 added
100731315 added
85975725 added
1321552596486807552 added


KeyboardInterrupt: 

## Building Network from Nodes

In [12]:
!pip install networkx

You should consider upgrading via the '/home/jacobbarkow/anaconda3/bin/python -m pip install --upgrade pip' command.


In [3]:
import networkx as nx

In [10]:
df = pd.read_csv('following_list.csv')
G = nx.from_pandas_edgelist(df, 'source', 'target')

In [11]:
G.number_of_nodes()

74379

In [18]:
# finding most central nodes

G_sorted = pd.DataFrame(sorted(G.degree, key=lambda x: x[1], reverse=True))
G_sorted.columns = ['nconst','degree']
top_central = G_sorted.head(20)

In [19]:
pd.options.mode.chained_assignment = None  # default='warn'

top_central['username'] = top_central.apply(lambda row: client.get_user(id = int(row.nconst)).data.username, axis = 1)
top_central['url'] = top_central.apply(lambda row: "https://twitter.com/" + str(row.username), axis = 1)

In [20]:
def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)

top_central.style.format({'url': make_clickable})

print(top_central)

          nconst  degree         username                                  url
0   5.608160e+08    1038   XiomaraCastroZ   https://twitter.com/XiomaraCastroZ
1   5.017718e+08    1029   danielmkovalik   https://twitter.com/danielmkovalik
2   3.188097e+09    1025     queeralamode     https://twitter.com/queeralamode
3   3.691263e+07    1022  agent_of_change  https://twitter.com/agent_of_change
4   7.133425e+17    1020     CarlosJRonVE     https://twitter.com/CarlosJRonVE
5   5.879555e+08    1018      shen_shiwei      https://twitter.com/shen_shiwei
6   1.101833e+08    1018   DanielDumbrill   https://twitter.com/DanielDumbrill
7   4.773062e+09    1017    ANTICONQUISTA    https://twitter.com/ANTICONQUISTA
8   1.427325e+08    1016   PlasenciaFelix   https://twitter.com/PlasenciaFelix
9   1.167697e+18    1015  ClaudiaJonesEdu  https://twitter.com/ClaudiaJonesEdu
10  8.114150e+17    1015  PeoplesParty_US  https://twitter.com/PeoplesParty_US
11  1.094276e+18    1014         AssalRad         ht